In [1]:
#####  LOADING AND EXTRACTING DATA   #####


import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle


#Modelo
def data_loader():
    
    # Load dataset file
    data_frame = pd.read_csv('model2/training.csv')
    data_frame.head()
    data_frame['Image'] = data_frame['Image'].apply(lambda i: np.fromstring(i, sep=' '))
    data_frame = data_frame.dropna()  # Get only the data with 15 keypoints
   
    # Extract Images pixel values
    imgs_array = np.vstack(data_frame['Image'].values)/ 255.0
    imgs_array = imgs_array.astype(np.float32)    # Normalize, target values to (0, 1)
    imgs_array = imgs_array.reshape(-1, 96, 96, 1)
        
    # Extract labels (key point cords)
    labels_array = data_frame[data_frame.columns[:-1]].values
    labels_array = (labels_array - 48) / 48    # Normalize, traget cordinates to (-1, 1)
    labels_array = labels_array.astype(np.float32) 
   
   
    # shuffle the train data
#     imgs_array, labels_array = shuffle(imgs_array, labels_array, random_state=9)  
    
    return imgs_array, labels_array

  


# # This snippet is just to check/verify data

#imgs, labels = data_loader()
#print(imgs.shape)
#print(labels.shape)
#print(labels)

#n=1
#labels[n] = (labels[n]*48)+48
#image = np.squeeze(imgs[n])
#plt.imshow(image, cmap='gray')
#plt.plot(labels[n][::2], labels[n][1::2], 'ro')
#plt.show()
    

In [2]:
######   BUILD, TRAIN AND SAVE THE CONVOLUTIONAL MODEL    ########


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Activation
from keras.layers import Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import ModelCheckpoint, History
# from keras.optimizers import Adam


# Main model
def the_model():
    model = Sequential()
    
    model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=X_train.shape[1:])) # Input shape: (96, 96, 1)
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    
    # Convert all values to 1D array
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(30))
    
    return model


X_train, y_train = data_loader()
print("Training datapoint shape: X_train.shape:{}".format(X_train.shape))
print("Training labels shape: y_train.shape:{}".format(y_train.shape))


epochs = 60
batch_size = 64

model = the_model()
hist = History()

checkpointer = ModelCheckpoint(filepath='checkpoint1.hdf5', 
                               verbose=1, save_best_only=True)

# Complie Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model_fit = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist], verbose=1)

model.save('model2/model1.h5')




Using TensorFlow backend.


Training datapoint shape: X_train.shape:(2140, 96, 96, 1)
Training labels shape: y_train.shape:(2140, 30)
Train on 1712 samples, validate on 428 samples
Epoch 1/60
1712/1712 [==============================] - 12s 7ms/step - loss: 0.0230 - accuracy: 0.5502 - val_loss: 0.0093 - val_accuracy: 0.4322

Epoch 00001: val_loss improved from inf to 0.00928, saving model to checkpoint1.hdf5
Epoch 2/60
1712/1712 [==============================] - 11s 7ms/step - loss: 0.0068 - accuracy: 0.6554 - val_loss: 0.0078 - val_accuracy: 0.4322

Epoch 00002: val_loss improved from 0.00928 to 0.00775, saving model to checkpoint1.hdf5
Epoch 3/60
1712/1712 [==============================] - 11s 6ms/step - loss: 0.0060 - accuracy: 0.6928 - val_loss: 0.0074 - val_accuracy: 0.4322

Epoch 00003: val_loss improved from 0.00775 to 0.00741, saving model to checkpoint1.hdf5
Epoch 4/60
1712/1712 [==============================] - 11s 7ms/step - loss: 0.0053 - accuracy: 0.7097 - val_loss: 0.0075 - val_accuracy: 0.4322



1712/1712 [==============================] - 11s 6ms/step - loss: 0.0010 - accuracy: 0.8493 - val_loss: 0.0036 - val_accuracy: 0.4930

Epoch 00037: val_loss did not improve from 0.00349
Epoch 38/60
1712/1712 [==============================] - 11s 6ms/step - loss: 9.8055e-04 - accuracy: 0.8610 - val_loss: 0.0035 - val_accuracy: 0.5561

Epoch 00038: val_loss improved from 0.00349 to 0.00349, saving model to checkpoint1.hdf5
Epoch 39/60
1712/1712 [==============================] - 11s 7ms/step - loss: 9.3223e-04 - accuracy: 0.8604 - val_loss: 0.0035 - val_accuracy: 0.5444

Epoch 00039: val_loss improved from 0.00349 to 0.00345, saving model to checkpoint1.hdf5
Epoch 40/60
1712/1712 [==============================] - 11s 6ms/step - loss: 8.8596e-04 - accuracy: 0.8540 - val_loss: 0.0034 - val_accuracy: 0.5584

Epoch 00040: val_loss improved from 0.00345 to 0.00339, saving model to checkpoint1.hdf5
Epoch 41/60
1712/1712 [==============================] - 11s 6ms/step - loss: 9.0717e-04 - acc